In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf, tensorflow.keras as keras
from keras.layers import Conv2D, Conv3D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization, Input
from keras.models import Model
import cv2
import os
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import scripts
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor

2024-03-17 21:14:27.879353: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-17 21:14:27.879408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-17 21:14:27.880278: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-17 21:14:27.887108: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-17 21:14:28.770310: W tensorflow/compiler/tf2

In [14]:
dfY = pd.read_csv("../data/train_labels.csv")[:20 * 100] # train on count number of image chains (each chain has 100 images)
# Generate path for each image (necessary for image generator)
dfY["path"] = [f"{id}/{i:03d}.png" for id, i in zip(dfY["chain_id"], dfY["i"])]

img = cv2.imread("../data/images/" + dfY.iloc[[5]].path.get(5))
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# find Harris corners
gray = np.float32(gray)
dst = cv2.cornerHarris(gray,2,3,0.04)
print(dst)
dst = cv2.dilate(dst, None)
# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]

cv2.imshow('dst',img)
if cv2.waitKey(0) & 0xff == 27:
    cv2.destroyAllWindows()

[[ 0.00921875  0.00921875  0.0053125  ... -0.00269531 -0.0009375
  -0.000625  ]
 [ 0.00921875  0.00921875  0.0053125  ... -0.00269531 -0.0009375
  -0.000625  ]
 [ 0.00921875  0.00921875  0.00707031 ...  0.02417969  0.01050781
  -0.000625  ]
 ...
 [ 0.00113281  0.00113281  0.00394531 ...  0.00160156  0.00347656
  -0.0025    ]
 [ 0.0259375   0.0259375   0.02441406 ...  0.00390625  0.00390625
   0.00097656]
 [ 0.00921875  0.00921875  0.02269531 ...  0.00101563  0.00347656
   0.000625  ]]
